In [1]:
import sys
import os

project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
    
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

from accelerate import notebook_launcher
from datasets import load_dataset

import logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)

INFO:datasets:PyTorch version 2.1.2 available.


In [2]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder="decoder_only",
    task_type="text_generation",
    inference_type="pure_generative",  # must match your ExperimentRunner expectations
    max_input_tokens=512,
    max_output_tokens=50,
    num_input_prompts=5,
    gpu_list=[0, 1],
    num_processes=2,
    batching_options={
        "fixed_max_batch_size": 2, # this is max batch size if adaptive batching on; fixed batch size if it's off
        "adaptive_batching": False,
        "adaptive_max_tokens": 256  
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1,
    fp_precision="float16",
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# Define a function to run the experiment.
def run_experiment():
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment, num_processes=experiment_config.num_processes)

Launching training on 2 GPUs.
Accelerator set up
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using backend pytorch with precision float16.


INFO:classes.experiment_runner:[Process 1406039] Model is on device: cuda:1


Model and tokenizer prepared
Original generate method reassigned


INFO:classes.experiment_runner:[Process 1406038] Model is on device: cuda:0
INFO:classes.experiment_runner:[Process 1406039] Dummy forward pass complete.
INFO:classes.experiment_runner:[Process 1406038] Dummy forward pass complete.


Allocated GPUs: GPU 0: NVIDIA A100-PCIE-40GB, GPU 1: NVIDIA A100-PCIE-40GB
Prompts processed: 5 prompts. Longest prompt has 71508 characters.
Energy tracking started
pure_generative task type
Fixed batching (non-adaptive): created 3 batches.


...

Energy tracking stopped


INFO:classes.experiment_runner:[Process 1406039] Inference complete.


Inference metrics saved
Aggregated energy metrics: []


[2025-03-22 11:38:30,279] torch.distributed.elastic.agent.server.api: [WARNING] Received Signals.SIGINT death signal, shutting down workers
[2025-03-22 11:38:30,280] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 1406039 via signal SIGINT


SignalException: Process 1405878 got signal: 2